In [19]:
from itertools import permutations

def candidate_set_generation(xi, pi0, Ti, T0i, F):
    # Initialize candidate sentence with masked tokens
    c0 = '[MASK]' * len(pi0)
    Ci = {c0}
    # Find shared POS tags between pi and pi0
    Tshared = Ti.intersection(T0i)
    for tk in Tshared:
        # Find words in xi that are tagged with tk
        Wk = [word for word, pos in xi if pos == tk]
        # Find positions in pi0 where tk occurs
        Sk = [i for i, pos in enumerate(pi0) if pos == tk]
        # Generate all possible assignments of words in Wk to positions in Sk
        Ak = list(permutations(Wk, len(Sk)))
        if not Ak:
            continue
        # Generate new candidate sentences by filling in masked tokens with word assignments
        new_Ci = set()
        for cj in Ci:
            for a in Ak:
                c0j = [c for c in cj]
                for i, word in zip(Sk, a):
                    c0j[i] = word
                new_Ci.add(' '.join(c0j))
        Ci = new_Ci
    # Apply mask-filling model to each candidate sentence and return the set of filled sentences
    return {F(cj) for cj in Ci}


In [ ]:
xi = [('I', 'PRP'), ('like', 'VBP'), ('fucking', 'NNS'), ('and', 'CC'), ('dogs', 'NNS')]
pi0 = ['PRP', 'VBP', 'NNS', 'CC', 'NNS']
Ti = {'PRP', 'VBP', 'NNS'}
T0i = {'PRP', 'VBP', 'NNS', 'CC'}
F = 

candidate_set_generation(xi, pi0, Ti, T0i, F)

{'I like     d o g s       f u c k i n g       S       K       ]       [       M       A       S       K       ]       [       M       A       S       K       ]       [       M       A       S       K       ]       [       M       A       S       K       ]',
 'I like     f u c k i n g       d o g s       S       K       ]       [       M       A       S       K       ]       [       M       A       S       K       ]       [       M       A       S       K       ]       [       M       A       S       K       ]'}

Distiled Roberta tokenizer

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 13.3 MB/s eta 0:00:00


In [3]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained("distilroberta-base")
tokenizer("Hello world")['input_ids']
tokenizer(" Hello world")['input_ids']  

[0, 20920, 232, 2]

In [7]:
# !pip install transformers.RobertaForMaSkedLM
!pip install transformers --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
from transformers import RobertaForMaskedLM

In [16]:
from transformers import RobertaTokenizer

In [17]:
import torch
# from transformers import RobertaTokenizer, RobertaForMaSkedLM

model = RobertaForMaskedLM.from_pretrained('distilroberta-base')
tokenizer = RobertaTokenizer.from_pretrained('distilroberta-base')

sequence = f"What the {tokenizer.mask_token} is going on?" # "The world will end in <mask>"

input_seq = tokenizer.encode(sequence, return_tensors='pt') # tensor([[0, 133, 232, 40, 253, 11, 50264, 2]])
mask_token_index = torch.where(input_seq == tokenizer.mask_token_id)[1] # (tensor([0]), tensor([6])) - we only want the the 2nd dimension
token_logits = model(input_seq).logits
masked_token_logits = token_logits[0, mask_token_index, :]

top_5_tokens = torch.topk(masked_token_logits, 5, dim=1).indices[0].tolist()

# print('sequence:', sequence)
# print('input_seq:', input_seq)
# print('mask_token_index:', mask_token_index)
# print('token_logits:', token_logits)
# print('masked_token_logits:', masked_token_logits)
print('top_5_tokens:', top_5_tokens)


top_5_tokens: [7105, 17835, 26536, 11141, 39072]


In [18]:
for token in top_5_tokens:
    print(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))

What the  hell is going on?
What the  heck is going on?
What the  fuck is going on?
What the  Hell is going on?
What the  HELL is going on?


In [ ]:
from itertools import permutations
Sk = [1,2,4,5]
Wk = ['A','B','C']
if len(Wk)<len(Sk):
    Ak = list(permutations(Sk,len(Wk)))
else:
    Ak = list(permutations(Wk,len(Sk)))
print(Ak)

In [31]:
from itertools import permutations
Sk = [1,2]
Wk = ['A','B']
if len(Wk)<len(Sk):
    Ak = list(permutations(Sk,len(Wk)))
else:
    Ak = list(permutations(Wk,len(Sk)))
print(Ak)

[('A', 'B'), ('B', 'A')]


In [ ]:
c0 = '[MASK]' * 4
Ci = {c0}

In [ ]:
Ci

In [ ]:
new_Ci = set()
for cj in Ci:
    for a in Ak:
        c0j = [c for c in cj]
        for i, word in zip(Sk, a):
            c0j[i] = word
        new_Ci.add(' '.join(c0j))
Ci = new_Ci

In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaForMaskedLM

# Load pre-trained RoBERTa model and tokenizer
model = RobertaForMaskedLM.from_pretrained('distilroberta-base')
tokenizer = RobertaTokenizer.from_pretrained('distilroberta-base')

# Set model to evaluation mode
model.eval()

# Define mask-filling function using RoBERTa
def fill_mask(text):
    # Tokenize input text
    input_ids = tokenizer.encode(text, return_tensors='pt')
    # Find the [MASK] token index
    mask_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1]
    # Get model predictions for the masked tokens
    with torch.no_grad():
        output = model(input_ids)
        print(output)
        mask_token_logits = output.logits[0, mask_token_index, :]
    # Get the top predicted tokens and convert them to strings
    top_tokens = torch.topk(mask_token_logits, k=5, dim=1).indices[0].tolist()
    top_token_strings = tokenizer.convert_ids_to_tokens(top_tokens)
    # Replace the [MASK] token with each of the top predicted tokens and return the resulting texts
    # print(top_token_strings)
    return [text[:mask_token_index[0].item()] + token + text[mask_token_index[0].item()+1:] for token in top_token_strings]

In [12]:
from itertools import permutations

def candidate_set_generation(xi, pi0, Ti, T0i, F):
    c0 = []
    for i in range(len(pi0)):
      c0.append('MASK')
    # c0 = ['MASK'] * len(pi0)
    print(c0)
    
    # Ci = {c0}
    Ci = c0
    
    Tshared = Ti.intersection(T0i)
    print(f'T shared {Tshared}')

    for tk in Tshared:
        # print(tk)

        Wk = [word for word, pos in xi if pos == tk]
        print(f'Wk {Wk}')
    
        Sk = [i for i, pos in enumerate(pi0) if pos[1] == tk]
        print(f'Sk {Sk}')

        # Ak = list(permutations(Wk, len(Sk)))
        if len(Wk)<=len(Sk):
            Ak = list(permutations(Sk,len(Wk)))
        else:
            Ak = list(permutations(Wk,len(Sk)))

          
        print(f'Ak {Ak}')

        if not Ak:
            continue
        new_Ci = set()
        for cj in Ci:
            print('cj',cj)
            for a in Ak:
                c0j = [c for c in cj]
                print('c0j', c0j)
                for i, word in zip(Sk, a):
                    print(i,word)
                    c0j[i] = word
                new_Ci.add(' '.join(c0j))
        Ci = new_Ci
        break
    print(len(Ci))
    return Ci
    # return {F(cj) for cj in Ci}


In [13]:
# non-offensive sentence
pi0 = 'it is not they happen all the time there are thousands of train derailments a year but when a particularly bad one happens news outlets jump on the trend and start reporting more they are not more frequent this is just the first a lot of people are hearing about them edit you all can downvote but i am right look it up if you are really concerned you should know this'
# non-offensive sentence pos tags 
tag_pi0 = ['PRP', 'VBZ', 'RB', 'PRP', 'VBP', 'PDT', 'DT', 'NN', 'EX', 'VBP', 'NNS', 'IN', 'NN', 'NNS', 'DT', 'NN', 'CC', 'WRB', 'DT', 'RB', 'JJ', 'NN', 'VBZ', 'NN', 'NNS', 'VB', 'IN', 'DT', 'NN', 'CC', 'VBP', 'VBG', 'JJR', 'PRP', 'VBP', 'RB', 'RBR', 'JJ', 'DT', 'VBZ', 'RB', 'DT', 'JJ', 'DT', 'NN', 'IN', 'NNS', 'VBP', 'VBG', 'IN', 'PRP', 'NN', 'PRP', 'DT', 'MD', 'VB', 'CC', 'PRP', 'VBP', 'JJ', 'VB', 'PRP', 'RP', 'IN', 'PRP', 'VBP', 'RB', 'JJ', 'PRP', 'MD', 'VB', 'DT']


# offensive sentence 
xi = 'i get this feeling that derailments happen a lot and we are only paying attention now because of what happened in ohio that said why the fuck do they happen so often'
# offensive sentence pos sequence 
tag_xi = ['PRP', 'VBP', 'DT', 'NN', 'IN', 'NNS', 'VBP', 'DT', 'NN', 'CC', 'PRP', 'VBP', 'RB', 'VBG', 'NN', 'RB', 'IN', 'IN', 'WP', 'VBD', 'IN', 'NNP', 'WDT', 'VBD', 'WRB', 'DT', 'BW', 'VBP', 'PRP', 'VB', 'RB', 'RB']


# unique pos tags/tokens of non-offensive sentence
T0i = set(['PRP', 'VBZ', 'RB', 'PRP', 'VBP', 'PDT', 'DT', 'NN', 'EX', 'VB"P', 'NNS', 'IN', 'NN', 'NNS', 'DT', 'NN', 'CC', 'WRB', 'DT', 'RB', 'JJ', 'NN', 'VBZ', 'NN', 'NNS', 'VB', 'IN', 'DT', 'NN', 'CC', 'VBP', 'VBG', 'JJR', 'PRP', 'VBP', 'RB', 'RBR', 'JJ', 'DT', 'VBZ', 'RB', 'DT', 'JJ', 'DT', 'NN', 'IN', 'NNS', 'VBP', 'VBG', 'IN', 'PRP', 'NN', 'PRP', 'DT', 'MD', 'VB', 'CC', 'PRP', 'VBP', 'JJ', 'VB', 'PRP', 'RP', 'IN', 'PRP', 'VBP', 'RB', 'JJ', 'PRP', 'MD', 'VB', 'DT'])
# unique pos tags/tokens of offensive sentence
Ti = set(['PRP', 'VBP', 'DT', 'NN', 'IN', 'NNS', 'VBP', 'DT', 'NN', 'CC', 'PRP', 'VBP', 'RB', 'VBG', 'NN', 'RB', 'IN', 'IN', 'WP', 'VBD', 'IN', 'NNP', 'WDT', 'VBD', 'WRB', 'DT', 'BW', 'VBP', 'PRP', 'VB', 'RB', 'RB'])


# tuple set of word in offensive sentence and its corresponding pos tag
xi_mod = [(word,tag) for word,tag in zip(xi.split(),tag_xi)]
# tuple set of word in non-offensive sentence and its corresponding pos tag
pi0_mod = [(word,tag) for word,tag in zip(pi0.split(),tag_pi0)]

In [14]:
print(candidate_set_generation(xi=xi_mod,pi0=pi0_mod,Ti=Ti,T0i=T0i,F=None))

['MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK', 'MASK']
T shared {'VBP', 'VBG', 'VB', 'NN', 'WRB', 'DT', 'NNS', 'RB', 'IN', 'PRP', 'CC'}
Wk ['get', 'happen', 'are', 'do']
Sk [4, 9, 30, 34, 47, 58, 65]
Ak [(4, 9, 30, 34), (4, 9, 30, 47), (4, 9, 30, 58), (4, 9, 30, 65), (4, 9, 34, 30), (4, 9, 34, 47), (4, 9, 34, 58), (4, 9, 34, 65), (4, 9, 47, 30), (4, 9, 47, 34), (4, 9, 47, 58), (4, 9, 47, 65), (4, 9, 58, 30), (4, 9, 58, 34), (4, 9, 58, 47), (4, 9, 58, 65), (4, 9, 65, 30), (4

IndexError: ignored

In [ ]:
# Example inputs
xi = [('The', 'DT'), ('fuck', '[MASK]'), ('brown', 'JJ'), ('fox', 'NN')]
pi = ['The', 'quick', 'brown', 'fox']
pi0 = ['DT', 'JJ', 'JJ', 'NN']
Vr = {'brown': 1}
Ti = {'DT', 'JJ', 'NN'}
T0i = {'DT', 'JJ', 'NN'}

# Call candidate_set_generation function
candidate_sentences = candidate_set_generation(xi, pi, pi0, Vr, Ti, T0i, fill_mask)

# Print the set of candidate sentences
print(candidate_sentences)


References

https://www.kaggle.com/code/juliusalphonso/filling-in-masked-words-with-roberta